In [ ]:
import mqr
from mqr.plot import Figure

In [ ]:
# Data and calculation libraries
import numpy as np
import pandas as pd
import scipy.stats as st

---
# A process converging to a new mean

In [ ]:
np.random.seed(234)

xs = np.linspace(0, 30, 30)[:, None]
ys = np.exp(-xs+20) / (1 + np.exp(-xs+20)) / 8 + 0.875 + st.norm().rvs([30, 5]) * 0.08

df = pd.DataFrame(
    data=ys,
    columns=['Obs1', 'Obs2', 'Obs3', 'Obs4', 'Obs5'])

In [ ]:
xbar = mqr.spc.XBarParams.from_data(df.iloc[:15, :])
stat = xbar.statistic(df)

with Figure(8, 3) as (fig, ax):
    mqr.plot.spc.chart(stat, xbar, ax)
    mqr.plot.spc.alarms(stat, xbar, mqr.spc.rules.limits, ax)
    
    # Mark the "historical" region in blue
    ax.axvspan(-0.5, 15.5, alpha=0.2, zorder=-1)

---
# A process with increasing variability
* Random samples of size 10, with increasing variance after the first 15 samples.
* The first 15 samples are used as historical data to calculate control limits.

In [ ]:
# Samples of size 10

np.random.seed(2345)

noise = np.full([30, 10], 10.0)
noise[:15] = st.norm(10.0, 0.01).rvs([15, 10])
for i in range(15, 30):
    factor = 1 + (i - 15) / 10
    noise[i, :] = st.norm(10, 0.01*factor).rvs([1, 10])
ys = noise

df = pd.DataFrame(
    data=ys,
    columns=[f'Obs{n}' for n in range(10)])

In [ ]:
with Figure(8, 3) as (fig, ax):
    ax.plot(df, linewidth=0, marker='.', color='C0')

In [ ]:
s = mqr.spc.SParams.from_data(df.iloc[:15, :])
stat = s.statistic(df)

with Figure(8, 3) as (fig, ax):
    mqr.plot.spc.chart(stat, s, ax)
    mqr.plot.spc.alarms(stat, s, mqr.spc.rules.limits, ax)

    # Mark the historical region with blue
    ax.axvspan(-0.5, 15.5, alpha=0.2, zorder=-1)

---
# An OC curve

In [ ]:
with Figure(8, 3) as (fig, ax):
    mqr.plot.spc.oc(n=40, c=6, defect_range=(0, 0.3), ax=ax)